# Question-answering system using Azure AI Search and Azure OpenAI

This notebook demonstrates how Azure AI Search and Azure OpenAI can be used together to build a simple question-answering system using the Retrieval-Augmented Generation (RAG) pattern. The system is built using the following Azure services:

- Azure AI Search
- Azure OpenAI

In a RAG pattern, queries and responses are coordinated between the search engine and the LLM. A user's question or query is forwarded to both the search engine and to the LLM as a prompt. The search results come back from the search engine and are redirected to an LLM. The response that makes it back to the user is generative AI, either a summation or answer from the LLM. [Learn more](https://learn.microsoft.com/azure/search/retrieval-augmented-generation-overview#content-retrieval-in-azure-ai-search).

### Prerequisites and install instructions
- Python 3.8+
- An Azure OpenAI service resource
    - Ensure you have the `Cognitive Services OpenAI User/Contributor` role assigned to your Azure user/identity on the Azure OpenAI service resource.
- An Azure AI Search service resource
    - Ensure you have the `Search Index Data User/Contributor` role assigned to your Azure user/identity on the Azure AI Search service resource.


### Install dependencies

In [ ]:
!pip install azure-search-documents==11.6.0b1 openai[datalib] azure-identity python-dotenv

### Import dependencies and load environment variables


In [2]:
import os

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI
import dotenv

dotenv.load_dotenv()


AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT")
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")

### Create a credential for authenticating

We will use the Azure Identity library to authenticate with Azure services. The library provides a set of credential classes that can be used to authenticate with Azure services. In this notebook, we will use the `DefaultAzureCredential` class, which will attempt authentication using multiple methods until it finds a valid credential.

In [3]:
credential = DefaultAzureCredential()

### Instantiate clients for Azure AI Search and Azure OpenAI

- `SearchClient` class from the `azure-search-documents` library for interacting with a search index in Azure AI Search.
- `AzureOpenAI` class from the `openai` library for interacting with the Azure OpenAI service.

In [4]:
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

openai_client = AzureOpenAI(
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider,
    api_version = "2024-02-01"
)
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=credential,
)

### Check how many documents are in the index

In [5]:
result = search_client.get_document_count()
print(result)

495


### Define method to retrieve the top documents from the index based on the user query

In [6]:
def search_index(query_text: str) -> list:
    results = search_client.search(search_text=query_text, top=3)

    documents = []
    for page in results.by_page():
        for document in page:
            documents.append(document)
    return documents

Let's validate that the method works by passing a sample query to it.

In [7]:
query_text = "Does PyCon require wearing a mask?"
documents = search_index(query_text)
for document in documents:
    print(document)


{'sourcefile': 'health-safety-guidelines.txt', 'sourcepage': 'health-safety-guidelines.txt', 'category': None, 'content': 'going to be doing some analysis on the airflow and refresh rates while we are in the convention center this year to see if there are some spaces that are well-ventilated enough to be indistinguishable from an outdoor situation for next PyCon US 2025. But for 2024, we are sticking with masking in all of the conference spaces.MASKS ARE REQUIREDWe are asking you to wear a mask during your time at PyCon US. That includes talks, the expo hall, hallways and sprints. We are looking into providing a second eating area outdoors for attendees who would like that – for either the airflow or the vitamin D or both.Exceptions are:Outdoor spacesIndoors while actively consuming food or beveragesSpeakers while presentingWhile necessary for communicating with someone who is hearing impaired when the ability to see the mouth is essential for communicationVery briefly to take photosMa

### Define the method to format the retrieved documents into a format that can be passed to the OpenAI API

In [8]:
def get_sources_content(results):
    sources = [
        (doc.get("sourcepage", "")) + ": " + doc.get("content", "").replace("\n", " ").replace("\r", " ")
        for doc in results
    ]
    content = "\n".join(sources)
    return content


Let's also validate that the method works by passing the documents retrieved in the previous step to it.

In [9]:
content = get_sources_content(documents)
print(content)

health-safety-guidelines.txt: going to be doing some analysis on the airflow and refresh rates while we are in the convention center this year to see if there are some spaces that are well-ventilated enough to be indistinguishable from an outdoor situation for next PyCon US 2025. But for 2024, we are sticking with masking in all of the conference spaces.MASKS ARE REQUIREDWe are asking you to wear a mask during your time at PyCon US. That includes talks, the expo hall, hallways and sprints. We are looking into providing a second eating area outdoors for attendees who would like that – for either the airflow or the vitamin D or both.Exceptions are:Outdoor spacesIndoors while actively consuming food or beveragesSpeakers while presentingWhile necessary for communicating with someone who is hearing impaired when the ability to see the mouth is essential for communicationVery briefly to take photosMasks must be worn over the nose and mouth and must be made of a tight-knit, non-permeable mate

### Define the method to generate an answer to a query using the OpenAI API

Here, we will construct a "conversation" using the documents retrieved from the search index and pass it to the OpenAI API to generate an answer to the user query.

In [10]:
def process_query(query_text: str) -> dict:
    documents = search_index(query_text)
    content = get_sources_content(documents)

    messages = [
        {
            "role": "user",
            "content": content

        },
        {
            "role": "assistant",
            "content": "Python is great!"
        },
        {
            "role": "user",
            "content": query_text
        }
    ]

    chat_completion = openai_client.chat.completions.create(
        model=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
        messages=messages,
        temperature=0.3,
        max_tokens=1024,
        n=1,
    )

    return chat_completion

### Submit a question and get an answer!

Using RAG, we can now submit a question to the system an answer enhanced with the search results.

In [11]:
query_text = "What FastAPI talks are there at PyCon?"
completion = process_query(query_text)
print(completion.choices[0].message.content)

At PyCon US 2024, one of the talks related to FastAPI is titled "Combining Django ORM & FastAPI in a Single App." This talk is scheduled for Saturday, May 18th, 2024, from 11:15 a.m. to 11:45 a.m. in Ballroom A and will be presented by Mia Bajić. The talk will focus on the practical implementation of combining Django ORM with FastAPI and share insights from the presenter's experience with this setup.
